## Deeplabv3  + Resnet101 CBAM_new

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import argparse
import os
import numpy as np
import sys
sys.argv = ['-f']

In [3]:
from train import *

In [4]:
parser = argparse.ArgumentParser(description="PyTorch DeeplabV3Plus Training")
parser.add_argument('--backbone', type=str, default='resnet',
                    choices=['resnet', 'xception', 'drn', 'mobilenet'],
                    help='backbone name (default: resnet)')
parser.add_argument('--out-stride', type=int, default=16,
                    help='network output stride (default: 8)')
parser.add_argument('--dataset', type=str, default='samsung_SEM',
                    choices=['pascal', 'coco', 'cityscapes','samsung_SEM','samsung_SEM_crop_256'],
                    help='dataset name (default: pascal)')
parser.add_argument('--use-sbd', action='store_true', default=False,
                    help='whether to use SBD dataset (default: False)')
parser.add_argument('--workers', type=int, default=0,

                    metavar='N', help='dataloader threads')
parser.add_argument('--base-size', type=int, default=1024,
                    help='base image size')
parser.add_argument('--crop-size', type=int, default=300,
                    help='crop image size')
parser.add_argument('--sync-bn', type=bool, default=None,
                    help='whether to use sync bn (default: auto)')
parser.add_argument('--freeze-bn', type=bool, default=False,
                    help='whether to freeze bn parameters (default: False)')
parser.add_argument('--loss-type', type=str, default='ce',
                    choices=['ce', 'focal'],
                    help='loss func type (default: ce)')
# training hyper params
parser.add_argument('--epochs', type=int, default=500, metavar='N',
                    help='number of epochs to train (default: auto)')
parser.add_argument('--start_epoch', type=int, default=0,
                    metavar='N', help='start epochs (default:0)')
parser.add_argument('--batch-size', type=int, default=16,
                    metavar='N', help='input batch size for \
                            training (default: auto)')
parser.add_argument('--test-batch-size', type=int, default=32,
                    metavar='N', help='input batch size for \
                            testing (default: auto)')
parser.add_argument('--use-balanced-weights', action='store_true', default=False,
                    help='whether to use balanced weights (default: False)')
# optimizer params
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: auto)')
parser.add_argument('--lr-scheduler', type=str, default='poly',
                    choices=['poly', 'step', 'cos'],
                    help='lr scheduler mode: (default: poly)')
parser.add_argument('--momentum', type=float, default=0.9,
                    metavar='M', help='momentum (default: 0.9)')
parser.add_argument('--weight-decay', type=float, default=5e-4,
                    metavar='M', help='w-decay (default: 5e-4)')
parser.add_argument('--nesterov', action='store_true', default=False,
                    help='whether use nesterov (default: False)')
# cuda, seed and logging
parser.add_argument('--no-cuda', action='store_true', default=
                    False, help='disables CUDA training')
parser.add_argument('--gpu-ids', type=str, default='0',
                    help='use which gpu to train, must be a \
                    comma-separated list of integers only (default=0)')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
# checking point
parser.add_argument('--resume', type=str, default=None,
                    help='put the path to resuming file if needed')
parser.add_argument('--checkname', type=str, default=None,
                    help='set the checkpoint name')
# finetuning pre-trained models
parser.add_argument('--ft', action='store_true', default=False,
                    help='finetuning on a different dataset')
# evaluation option
parser.add_argument('--eval-interval', type=int, default=1,
                    help='evaluuation interval (default: 1)')
parser.add_argument('--no-val', action='store_true', default=False,
                    help='skip validation during training')

args = parser.parse_args()

args.backbone_pretrained = None
args.cuda = not args.no_cuda and torch.cuda.is_available()
if args.cuda:
    try:
        args.gpu_ids = [int(s) for s in args.gpu_ids.split(',')]
    except ValueError:
        raise ValueError('Argument --gpu_ids must be a comma-separated list of integers only')
        
args.useCPU = False
args.batch_size =2
args.lr = 0.001
#args.base_size = 400
#args.crop_size = 400
args.epochs = 5
args.seed = 999
args.out_stride = 16

#args.dataset = 'samsung_CAD_BE_crop_256_NormalRatio_1'
#args.dataset = 'samsung_CAD_BE_crop_512_NormalRatio_1'
args.dataset = 'samsung_CAD_BE_1024_margin'

args.checkname = 'resnet101_CBAM_1024_new_model_margin'
#args.checkname = 'resnet101_CBAM_256_new_model'
#args.checkname = 'resnet101_CBAM_512_new_model'

args.backbone = 'resnet101_CBAM' #xception
# args.backbone = 'resnet101'

args.val_ref_path = 'E:/3_interdata/1009+1111+0927/0.generatedData/CAD'
#args.val_ref_path = '/data2/0_Samsung_Data/3_Train_Data/Hotspot_prediction_using_CAD/DataSet_1_Size_1024x1024_margin/CAD+SEM_Images_with_box'
#args.val_ref_path = '/data2/0_Samsung_Data/3_Train_Data/Hotspot_prediction_using_CAD/DataSet_1_Size_256x256_NormalRatio_1/CAD+SEM_Images_with_box'
#args.val_ref_path = '/data2/0_Samsung_Data/3_Train_Data/Hotspot_prediction_using_CAD/DataSet_1_Size_512x512_NormalRatio_1/CAD+SEM_Images_with_box'

args.GroupNorm = False
args.loss_type = 'mse'

args.save_folder ='E:/4_CAD_based_Hotspot_detection/Tensorboard_files/unsupervise'

#args.backbone_pretrained = '/data2/2_ML_team/jaihoons/1_Research/99_Tensorboard_files/'
#args.backbone_pretrained += '1_Classification/samsung_CAD_BE_crop_512_NormalRatio_1/resnet50_CBAM_512_NR_1/'
#args.backbone_pretrained += 'model_best.pth'

# args.resume = '/data2/2_ML_team/ml_ty/4_CAD_based_Hotspot_detection/3_Tensorboard_files/'
# args.resume = 'E:/4_CAD_based_Hotspot_detection/Tensorboard_files/paper_iccad/samsung_CAD_BE_1024_margin/resnet101_CBAM_1024_new_model_margin/experiment_0/'
# args.resume += 'checkpoint.pth.tar'

#args.resume = '/data2/2_ML_team/jaihoons/1_Research/99_Tensorboard_files/'
#args.resume += '2_Segmentation/samsung_CAD_BE_crop_256_NormalRatio_1/resnet50_CBAM_256_new_model/'
#args.resume += 'model_best.pth.tar'

#args.resume = '/data2/2_ML_team/jaihoons/1_Research/99_Tensorboard_files/'
#args.resume += '1_Classification/samsung_CAD_BE_crop_512_NormalRatio_1/resnet50_CBAM_512_NR_1/'
#args.resume += 'model_best.pth'

#args.resume = '/data2/2_ML_team/jaihoons/1_Research/99_Tensorboard_files/'
#args.resume += '2_Segmentation/samsung_CAD_BE_crop_256_NormalRatio_1/resnet50_CBAM_256/experiment_0/'
#args.resume += 'checkpoint.pth.tar'

#args.resume = '/data2/2_ML_team/jaihoons/1_Research/99_Tensorboard_files/
#args.resume += '2_Segmentation/samsung_CAD_BE_crop_256_NormalRatio_1/resnet50_CBAM_256/experiment_0#args.resume += 'checkpoint.pth.tar'

# args.resume = 'E:/4_CAD_based_Hotspot_detection/Tensorboard_files/'
# args.resume += '1022+1030+1031/samsung_CAD_BE_1024_margin/resnet101_CBAM_1024_new_model_margin/'
# args.resume += 'experiment_7/checkpoint.pth.tar'

args.optimizer = 'Adam'
args.gpu_ids = 0
args.ft = True

args.no_val = False
args.eval_interval = 5

print(args)

Namespace(GroupNorm=False, backbone='resnet101_CBAM', backbone_pretrained=None, base_size=1024, batch_size=2, checkname='resnet101_CBAM_1024_new_model_margin', crop_size=300, cuda=True, dataset='samsung_CAD_BE_1024_margin', epochs=5, eval_interval=5, freeze_bn=False, ft=True, gpu_ids=0, loss_type='mse', lr=0.001, lr_scheduler='poly', momentum=0.9, nesterov=False, no_cuda=False, no_val=False, optimizer='Adam', out_stride=16, resume=None, save_folder='E:/4_CAD_based_Hotspot_detection/Tensorboard_files/unsupervise', seed=999, start_epoch=0, sync_bn=None, test_batch_size=32, useCPU=False, use_balanced_weights=False, use_sbd=False, val_ref_path='E:/3_interdata/1009+1111+0927/0.generatedData/CAD', weight_decay=0.0005, workers=0)


## Model Test

In [10]:
import torch
import tensorwatch as tw

In [11]:
import modeling.backbone.resnet_CBAM as resnet_CBAM

In [12]:
CBAM_model = resnet_CBAM.ResNet50(16, torch.nn.InstanceNorm2d, pretrained=False)

In [13]:
CBAM_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size

In [14]:
tw.draw_model(CBAM_model,[1,3,512,512])

AttributeError: module 'torch.onnx' has no attribute 'set_training'

## Dataset Test

In [ ]:
from dataloaders import make_data_loader
kwargs = {'num_workers': args.workers, 'pin_memory': True}
train_loader, val_loader, test_loader, nclass = make_data_loader(args, **kwargs)


In [ ]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
def show(img):
    npimg = img.numpy()
    fig = plt.figure(figsize=(25,5))
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    
sample = next(iter(train_loader))

image,  insert_CAD ,target = sample['image'], sample['insert_CAD'], sample['label']
image_name = sample['image_name']

In [ ]:
print(image_name)

In [ ]:
show(make_grid(image, padding=10, normalize=True))

In [ ]:
show(make_grid(insert_CAD, padding=10, normalize = True))

In [ ]:
show(make_grid(target, padding=10, normalize=True))

### Training and Validation###

In [ ]:

import time

torch.manual_seed(args.seed)
trainer = Trainer(args)
print('Starting Epoch:', trainer.args.start_epoch)
print('Total Epoches:', trainer.args.epochs)
time.sleep(1)

for epoch in range(trainer.args.start_epoch, trainer.args.epochs):
    trainer.training(epoch)
    if not trainer.args.no_val and epoch % args.eval_interval == (args.eval_interval - 1):
        trainer.validation(epoch, val_ref_image_path=args.val_ref_path)
trainer.writer.close()

In [ ]:
torch.cuda.empty_cache()

### Only Validation ###

In [6]:
import time
args.batch_size = 1
args.dataset = 'samsung_CAD_BE_1024_margin'
args.checkname = 'E:/4_CAD_based_Hotspot_detection/Tensorboard_files/Resnet50_Nonlocal_deeplab/samsung_CAD_BE_1024_margin/TEST3_samsung_deeplab_cad_heatmap_xception_800x800gpu_03'

args.resume ='E:/4_CAD_based_Hotspot_detection/Tensorboard_files/1022+1030+1031/' \
             'samsung_CAD_BE_1024_margin/resnet101_CBAM_1024_new_model_margin/experiment_7/' \
            
args.resume += 'checkpoint.pth.tar'
# args.resume ='E:/4_CAD_based_Hotspot_detection/Tensorboard_files/paper_iccad/' \
#              'samsung_CAD_BE_1024_margin/resnet101_CBAM_1024_new_model_margin/'
# args.resume += 'model_best.pth.tar'

#args.resume = 'E:/4_CAD_based_Hotspot_detection/Tensorboard_files' \
#              '/1022+1030+1031/samsung_CAD_BE_1024_margin/resnet101_CBAM_1024_new_model_margin/experiment_7/checkpoint.pth.tar'
args.GroupNorm = False

torch.manual_seed(args.seed)
trainer = Trainer(args)
print('Starting Epoch:', trainer.args.start_epoch)
print('Total Epoches:', trainer.args.epochs)
time.sleep(1)
trainer.validation(0, use_data='test',
                   val_ref_image_path= 'E:/3_interdata/1009+1111+0927/CAD')
trainer.writer.close()

Using device: cuda:0
GPU CARD 0
GeForce RTX 3080
Memory Usage:
Allocated: 1.1 GB
Cached:    1.6 GB
E:/5_weaklycontour/0_data/1_SamsungContour
Number of images in train: 1900
E:/5_weaklycontour/0_data/1_SamsungContour
Number of images in test: 270
Norm layer:  <class 'torch.nn.modules.instancenorm.InstanceNorm2d'>
=> loaded checkpoint 'E:/4_CAD_based_Hotspot_detection/Tensorboard_files/1022+1030+1031/samsung_CAD_BE_1024_margin/resnet101_CBAM_1024_new_model_margin/experiment_7/checkpoint.pth.tar' (epoch 3)
reset best pred..
Starting Epoch: 0
Total Epoches: 5



C:\Users\Sonic\Anaconda3\lib\site-packages\torch\cuda\memory.py:375: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


ValueError: Expected more than 1 spatial element when training, got input size torch.Size([1, 256, 1, 1])

### TEST or Inference ###

In [ ]:
import time
from test import *

args.useCPU = False
args.dataset = 'samsung_CAD_BE'
args.batch_size = 10
args.ft = True
args.gpu_ids = 0
args.start_epoch = 0
args.epochs = 1
args.seed = 1423
args.backbone = 'xception'
args.GroupNorm = True
args.checkname = 'TEST_1k_samsung_deeplab_cad_heatmap_xception_800x800:gpu_00'

args.resume = '/data/2_ML_team_data/jaihoons/1_SamsungProject/3_Tensor_Board_Files/2_Samsung_CAD/'
args.resume += 'samsung_CAD_BE_crop_800/samsung_deeplab_cad_heatmap_xception_800x800:gpu_03/'
args.resume += 'model_best.pth.tar'

#CAD_image_dir = '/data/1_data/Samsung_Image_Data/blind_test_result/case2/CAD'
#SEM_image_dir = '/data/1_data/Samsung_Image_Data/blind_test_result/case2/Image'

CAD_image_dir = '/data/1_data/Samsung_Image_Data/images/blind_image_data_one_NEW/CAD'
SEM_image_dir = '/data/1_data/Samsung_Image_Data/images/blind_image_data_one_NEW/CAD+Image'
list_txt = '/data/1_data/Samsung_Image_Data/images/blind_image_data_one_NEW/test.txt'

#CAD_image_dir = '/data/1_data/Samsung_Image_Data/images/blind_image_data_one_NEW_VOC_style_800x800/CAD_Images'
#SEM_image_dir = '/data/1_data/Samsung_Image_Data/images/blind_image_data_one_NEW_VOC_style_800x800/CAD+SEM_Images_with_box'
#list_txt = '/data/1_data/Samsung_Image_Data/images/blind_image_data_one_NEW_VOC_style_800x800/ImageSets/Main/test.txt'

torch.manual_seed(args.seed)
tester = Tester(args, CAD_image_dir=CAD_image_dir, SEM_image_dir=SEM_image_dir,list_txt=list_txt, nclass=1)
#-----------------------
start_time = time.time()
tester.test_batch(only_extract_result_img=True)
tester.writer.close()
end_time = time.time()
#-----------------------
print("Elaped Time: %s seconds" % ( end_time - start_time))